In [ ]:
# Importação das bibliotecas

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Carregamento do dataset

pd.set_option('display.max_rows', None)

# O método load_dataset carrega um arquivo para trabalharmos. O nome do arquivo deve ser um dos nomes que está no seguinte repositório: https://github.com/mwaskom/seaborn-data
# O método copy() faz uma cópia do dataset
df = sns.load_dataset("tips").copy()
# df = pd.read_csv("tips.csv").copy()

# O método head() retorna por padrão as 5 primeiras linhas do arquivo. O parâmetro opcional n indica a quantidade de linhas que queremos que retorne
df.head()

# O método tail funciona de maneira semelhante, porém retorna as últimas linhas do arquivo.
df.tail()

# O método to_csv salva o arquivo carregado com load_dataset localmente no formato csv.
df.to_csv("tips.csv")

In [ ]:
# Características do data frame

# O atributo shape retorna uma tupla contendo a quantidade de linhas e de colunas
print(f"Shape: {df.shape}.")

# O atributo dtypes retorna os tipos de dados das colunas do dataframe.
print(f"Colunas e tipos: {df.dtypes}.")

print(f"\nResumo estatístico (numérico):")
display(df.describe(include="all").T)

In [ ]:
# Contagem dos valores e faltantes
print("Valores ausentes por coluna:")

# O método isna() gera um novo dataframe a partir do atual. Com a seguinte diferença: Esse novo dataframe terá apenas valores booleanos (True ou False). Caso o valor da coluna não esteja preenchido, esse valor retornado será True. Caso o dado tenha sido preenchido, retornará False.
df.isna()

# O método sum() aplicado no dataframe gerado pelo método isna(), fará a soma dos valores das colunas do dataframe. No nosso caso, será retornado uma nova Serie com os valores zerados, pois não há dados faltantes no dataframe tips
display(df.isna().sum())

# O método sum() retorna um objeto do tipo Series. Basicamente um objeto desse tipo contém apenas 1 dimensão, enquanto o Dataframe contém 2 dimensões. Podemos entender um objeto Series como uma lista mais robusta.

print("\nContagem por categoria")

print(f"{df['day'].value_counts()}\n")
print(f"{df['time'].value_counts()}\n")
print(f"{df['smoker'].value_counts()}\n")
print(f"{df['sex'].value_counts()}\n")

In [ ]:
# Limpeza dos dados

# O método to_numeric tenta converter o valor encontrado em um tipo numérico. O parâmetro coerce faz com que não seja gerado um erro caso o tipo encontrado seja imcompatível com o tipo numérico
df["total_bill"] = pd.to_numeric(df["total_bill"], errors="coerce")
df["tip"] = pd.to_numeric(df["tip"], errors="coerce")

print("NANs após tratamento")
# A linha abaixo mostra quantos valores NAN existem nas colunas total_bill e tip. NAN significa Not A Number.
display(df[["total_bill", "tip"]].isna().sum())

# df.head()

In [ ]:
# Feature Engineering (criar uma nova coluna): tip_percent (percentual da gorjeta)

# O cálculo feito abaixo é aplicado em todas as linhas do dataframe, não sendo necessário o uso do loop for.
# É importance mencionar que caso o valor de tip ou total_bill seja um NAN, o valor de tip_percent também será.
df["tip_percent"] = (df["tip"] / df["total_bill"]) * 100

# O método round(2) indica a quantidade máxima de casas decimais do valor.
df["tip_percent"] = df["tip_percent"].round(2)

# Exibimos os valores de total_bill, tip e tip_percent
df[["total_bill", "tip", "tip_percent"]].head()